# Checkpoint 1

A partir do dataset enviado, responda:

## Importação e Análise Inicial

1. Importe o arquivo CSV.(0,33)
2. Quantas linhas tem o dataset?(0,33)
3. Quantas colunas tem o dataset?(0,33)
4. Quais colunas têm o tipo `object`?(0,34)
5. Quais colunas têm o tipo `int64`?(0,34)
6. Existe alguma coluna de outro tipo?(0,33)

## Análise Estatística

7. Qual foi o total, o máximo, o mínimo do objetivo financeiro de todos os projetos (goal)?(0,66)
8. Qual o total, o máximo, o mínimo e o valor que representa 25% do dinheiro levantado (pledged)?(0,67)
9. Qual o total, o máximo, o mínimo e o valor que representa 75% dos apoiadores dos projetos(backers)?(0,67)

## Análise de Categorias

10. Qual a `category` mais presente? E quantos valores únicos ela possui?(0,5)
11. Qual a `main_category` mais presente? E quantos valores únicos ela possui?(0,5)
12. Qual a `currency` mais presente? E quantos valores únicos ela possui?(0,5)
13. Execute o processo de dummy na coluna `country`.(0,5)

## Análise de Valores Nulos

14. Qual a quantidade de valores nulos em cada feature?(0,66)
15. Qual a proporção de valores nulos por valores não nulos em cada feature?(0,66)
16. Faça o tratamento desses nulos e explique por que escolheu cada método.(0,68)

## Visualização Gráfica

17. Visualização Gráfica da feature `state`.(2)



----------------------------------------
🧩 Desafio: ETL de Dados Públicos

🎯 Objetivo
Construir um pipeline ETL no Apache Airflow que:

Extrai dados de algum dataset público

Transforma os dados limpando, filtrando e unificando informações.

Carrega o resultado em um banco de dados PostgreSQL (ou salva como Parquet).

Agenda a execução diária do pipeline (simulando ingestão incremental).

# Checkpoint 1

A partir do dataset enviado, responda:

## Importação e Análise Inicial

1. Importe o arquivo CSV.(0,33)
2. Quantas linhas tem o dataset?(0,33)
3. Quantas colunas tem o dataset?(0,33)
4. Quais colunas têm o tipo `object`?(0,34)
5. Quais colunas têm o tipo `int64`?(0,34)
6. Existe alguma coluna de outro tipo?(0,33)

In [1]:
import pandas as pd
import numpy as np
import ast

# Gerenciamento de experimentos com MLflow
import mlflow

# Download do dataset de filmes do Kaggle
#!kaggle datasets download -d rounakbanik/the-movies-dataset -p ./movies_dataset

# Descompactar o arquivo baixado
#!unzip -o ./movies_dataset/the-movies-dataset.zip -d ./movies_dataset


# Carregar o dataset de filmes usando o motor 'python' para evitar o ParserError
try:
    df = pd.read_csv("./movies_dataset/movies_metadata.csv", low_memory=False)
    print("O dataset foi importado com sucesso!")

    # Obter o número de linhas e colunas do DataFrame
    numero_linhas, numero_colunas = df.shape
    print(f"O dataset tem {numero_linhas} linhas")
    print(f"O dataset tem {numero_colunas} colunas.")

    colunas_object= df.select_dtypes(include=['object'])
    colunas_int64 = df.select_dtypes(include=['int64'])
    colunas_outrotipo = df.select_dtypes(exclude=['object', 'int64'])
    print(f"As colunas do tipo 'object' são: {colunas_object.columns.tolist()}")
    print(f"As colunas do tipo 'int64' são: {colunas_int64.columns.tolist()}")
    print(f"As colunas de outro tipo são: {colunas_outrotipo.columns.tolist()}")
    
except Exception as e:
    print(f"Ocorreu um erro ao carregar o CSV: {e}")
    print("Verifique se o arquivo 'mymoviedb.csv' existe no diretório './movies_dataset' e não está corrompido.")


O dataset foi importado com sucesso!
O dataset tem 45466 linhas
O dataset tem 24 colunas.
As colunas do tipo 'object' são: ['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'spoken_languages', 'status', 'tagline', 'title', 'video']
As colunas do tipo 'int64' são: []
As colunas de outro tipo são: ['revenue', 'runtime', 'vote_average', 'vote_count']


## Análise Estatística

7. Qual foi o total, o máximo, o mínimo do objetivo financeiro de todos os projetos (goal)?(0,66)
8. Qual o total, o máximo, o mínimo e o valor que representa 25% do dinheiro levantado (pledged)?(0,67)
9. Qual o total, o máximo, o mínimo e o valor que representa 75% dos apoiadores dos projetos(backers)?(0,67)

In [2]:
try:

    # --- Limpeza e Conversão da Coluna 'budget' ---
    # Converte a coluna 'budget' para um tipo numérico.
    # Erros de conversão (valores não numéricos) serão transformados em NaN (Not a Number).
    df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

    # Filtra o DataFrame para incluir apenas filmes com orçamento maior que zero.
    filmes_com_budget = df[df['budget'] > 0].copy()

    # --- Análise Estatística ---
    total_budget = filmes_com_budget['budget'].sum()
    max_budget = filmes_com_budget['budget'].max()
    min_budget = filmes_com_budget['budget'].min()

    print(f"Total do orçamento de todos os filmes: ${total_budget:,.2f}")
    print(f"Maior orçamento entre os filmes: ${max_budget:,.2f}")
    print(f"Menor orçamento entre os filmes: ${min_budget:,.2f}")


    # --- Limpeza e Conversão da Coluna 'revenue' ---
    # A coluna 'revenue' também vem como texto e precisa ser convertida para número.
    # Usamos 'errors='coerce'' para transformar valores inválidos em NaN (Not a Number).
    df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')

    # Filtra o DataFrame para incluir apenas filmes com receita maior que zero.
    filmes_com_revenue = df[df['revenue'] > 0].copy()

    # --- Análise Estatística ---
    estatisticas_revenue = filmes_com_revenue['revenue'].describe()

    # Extraindo os valores do resultado do .describe()
    min_revenue = estatisticas_revenue['min']
    max_revenue = estatisticas_revenue['max']
    percentil_25 = estatisticas_revenue['25%']

    # Cálculo da receita total (o describe() não fornece a soma diretamente)
    total_revenue = filmes_com_revenue['revenue'].sum()

    print(f"Total da receita de todos os filmes: ${total_revenue:,.2f}")
    print(f"O maior valor de receita entre os filmes: ${max_revenue:,.2f}")
    print(f"O menor valor de receita entre os filmes: ${min_revenue:,.2f}")
    print(f"O valor do 25º percentil da receita: ${percentil_25:,.2f}")
    

    # --- Limpeza e Conversão da Coluna 'vote_count' ---
     # A coluna 'vote_count' também vem como texto e precisa ser convertida para número.
    # Usamos 'errors='coerce'' para transformar valores inválidos em NaN (Not a Number).
    df['vote_count'] = pd.to_numeric(df['vote_count'], errors='coerce')

    # Filtra o DataFrame para incluir apenas filmes com contagem de votos maior que zero.
    filmes_com_vote_count = df[df['vote_count'] > 0].copy()

    # --- Análise Estatística ---
    estatisticas_vote_count = filmes_com_vote_count['vote_count'].describe()

    # Extraindo os valores do resultado do .describe()
    min_vote_count = estatisticas_vote_count['min']
    max_vote_count = estatisticas_vote_count['max']
    percentil_75 = estatisticas_vote_count['75%']

    # Cálculo da receita total (o describe() não fornece a soma diretamente)
    total_vote_count = filmes_com_vote_count['vote_count'].sum()

    print(f"Total da contagem de votos de todos os filmes: {total_vote_count:,}")
    print(f"O maior valor de contagem de votos entre os filmes: {max_vote_count:,}")
    print(f"O menor valor de contagem de votos entre os filmes: {min_vote_count:,}")
    print(f"O valor do 75º percentil da contagem de votos: {percentil_75:,}")

except Exception as e:
    print(f"Ocorreu um erro durante a análise: {e}") 


Total do orçamento de todos os filmes: $192,062,026,597.00
Maior orçamento entre os filmes: $380,000,000.00
Menor orçamento entre os filmes: $1.00
Total da receita de todos os filmes: $509,576,984,595.00
O maior valor de receita entre os filmes: $2,787,965,087.00
O menor valor de receita entre os filmes: $1.00
O valor do 25º percentil da receita: $2,400,000.00
Total da contagem de votos de todos os filmes: 4,995,933.0
O maior valor de contagem de votos entre os filmes: 14,075.0
O menor valor de contagem de votos entre os filmes: 1.0
O valor do 75º percentil da contagem de votos: 38.0


## Análise de Categorias

10. Qual a `category` mais presente? E quantos valores únicos ela possui?(0,5)

In [3]:
try:
    # Remove linhas com valores NaN na coluna 'genres'
    df.dropna(subset=['genres'], inplace=True)

    # Aplica a função para extrair os nomes dos gêneros
    def extrair_nomes_generos(generos_str):
        try:
            # Converte a string para uma lista de dicionários usando ast.literal_eval
            generos_lista = ast.literal_eval(generos_str)

            # Extrai os nomes dos gêneros da lista de dicionários
            nomes_generos = [genero['name'] for genero in generos_lista]
            return nomes_generos
        except (ValueError, SyntaxError):
            # Em caso de erro na conversão, retorna uma lista vazia
            return []
    
    # Aplica a função à coluna 'genres' e cria uma nova coluna 'generos_nomes'
    df['generos_nomes'] = df['genres'].apply(extrair_nomes_generos)


    # Cria uma lista com todos os gêneros presentes no DataFrame
    todos_os_generos = []
    for lista in df['generos_nomes']:
        todos_os_generos.extend(lista)

    # Conta a frequência de cada gênero
    contagem_generos = pd.Series(todos_os_generos).value_counts()

    # Obtém o gênero mais frequente e o número de gêneros únicos
    genero_mais_frequente = contagem_generos.index[0]
    
    # Número de gêneros únicos
    numero_generos_unicos = len(contagem_generos)

    print(f"O gênero mais frequente é: {genero_mais_frequente}")
    print(f"Número de gêneros únicos: {numero_generos_unicos}")

except KeyError as ke:
    print(f"A coluna 'genres' não foi encontrada no DataFrame: {ke}")
except Exception as e:
    print(f"Ocorreu um erro ao processar a coluna 'genres': {e}")


O gênero mais frequente é: Drama
Número de gêneros únicos: 32


11. Qual a `main_category` mais presente? E quantos valores únicos ela possui?(0,5) --> considerando que o indice 0 da lista é a categoria principal

In [4]:
try: 
    # Esta função extrai APENAS o primeiro gênero da lista.
    # Considerando o que o primeiro gênero é o principal.
    def extrair_genero_principal(generos_str):
        try:
            # Converte a string em uma lista de dicionários
            lista_generos = ast.literal_eval(generos_str)
            
            # Verifica se a lista não está vazia antes de pegar o primeiro item
            if lista_generos:
                # Retorna o nome do primeiro gênero (índice 0)
                return lista_generos[0]['name']
            else:
                # Retorna None se a lista de gêneros estiver vazia
                return None
        except (ValueError, SyntaxError):
            # Retorna None se a string for malformada
            return None

    # Aplica a função à coluna 'genres' para criar a nova coluna 'genero_principal'
    df['genero_principal'] = df['genres'].apply(extrair_genero_principal)

    # Conta a frequência de cada gênero principal
    contagem_genero_principal = df['genero_principal'].value_counts()

    # Obtém o gênero principal mais frequente e sua contagem
    genero_principal_mais_frequente = contagem_genero_principal.index[0]
    contagem_do_mais_frequente = contagem_genero_principal.iloc[0]

    print(f"O gênero principal mais frequente é: {genero_principal_mais_frequente}")
    print(f"Número de filmes com esse gênero principal: {contagem_do_mais_frequente}")

except KeyError as ke:
    print(f"A coluna 'genres' não foi encontrada no DataFrame: {ke}")
except Exception as e:
    print(f"Ocorreu um erro ao processar a coluna 'genres': {e}")


O gênero principal mais frequente é: Drama
Número de filmes com esse gênero principal: 11966


12. Qual a `currency` mais presente? E quantos valores únicos ela possui?(0,5) --> substituido pelo `original_language`

In [5]:
try:
    # Análise da coluna 'original_language'
    # Considerando que o currency foi substituído por 'original_language'
    contagem_idioma_mais_frequente = df['original_language'].dropna().value_counts()

    # Obtém o idioma mais frequente e sua contagem
    idioma_mais_frequente = contagem_idioma_mais_frequente.index[0]
    contagem_idioma = contagem_idioma_mais_frequente.iloc[0]

    print(f"O idioma original mais frequente é: {idioma_mais_frequente}")
    print(f"Número de filmes com esse idioma original: {contagem_idioma}")

except KeyError as ke:
    print(f"A coluna 'original_language' não foi encontrada no DataFrame: {ke}")
except Exception as e:
    print(f"Ocorreu um erro ao processar a coluna 'original_language': {e}")


O idioma original mais frequente é: en
Número de filmes com esse idioma original: 32269


13. Execute o processo de dummy na coluna `country`.(0,5) --> substituido por `production_countries`

In [6]:
try:
    # --- ETAPA 1: Extração e Criação das Dummies ---
    
    # É uma boa prática trabalhar em uma cópia para evitar SettingWithCopyWarning
    # e manter o DataFrame original intacto, se necessário.
    df_processado = df.copy()

    # Remove linhas onde a coluna é nula.
    df_processado.dropna(subset=['production_countries'], inplace=True)

    # Função para extrair os nomes dos países
    def extrair_nomes_paises(paises_str):
        # A verificação de tipo já garante que não é NaN, mas mantemos por robustez
        if not isinstance(paises_str, str) or paises_str.strip() in ('', '[]'):
            return []
        try:
            lista_paises = ast.literal_eval(paises_str)
            if isinstance(lista_paises, list):
                return [item['name'] for item in lista_paises if isinstance(item, dict) and 'name' in item]
            return []
        except (ValueError, SyntaxError):
            return []

    # Aplica a função para obter listas de países
    listas_de_paises = df_processado['production_countries'].apply(extrair_nomes_paises)

    # Cria as colunas dummy em um DataFrame separado
    paises_dummies = listas_de_paises.str.join('|').str.get_dummies()
    
    # Remove a coluna vazia '' se ela for criada
    if '' in paises_dummies.columns:
        paises_dummies = paises_dummies.drop(columns=[''])

    print(f"Foram criadas {paises_dummies.shape[1]} novas colunas de países.")
    
    # --- ETAPA 2: Juntar as novas colunas ao DataFrame ---
    
    # Concatena o DataFrame processado com as novas colunas dummy.
    # O 'axis=1' indica que a junção é por colunas.
    # Usamos os índices para garantir que as linhas corretas sejam combinadas.
    df_tratado = pd.concat([df_processado, paises_dummies], axis=1)

    print("\nNovas colunas foram adicionadas ao DataFrame!")
    print("\nAmostra do DataFrame final com as 5 primeiras colunas dummy:")
    
    # Mostra colunas originais e as primeiras 5 novas colunas
    colunas_para_mostrar = list(df.columns) + list(paises_dummies.columns[:5])
    print(df_tratado[colunas_para_mostrar].head())

except KeyError as ke:
    print(f"A coluna 'production_countries' não foi encontrada no DataFrame: {ke}")
except Exception as e:
    print(f"Ocorreu um erro ao processar a coluna 'production_countries': {e}")



Foram criadas 160 novas colunas de países.

Novas colunas foram adicionadas ao DataFrame!

Amostra do DataFrame final com as 5 primeiras colunas dummy:
   adult                              belongs_to_collection      budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000.0   
1  False                                                NaN  65000000.0   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0.0   
3  False                                                NaN  16000000.0   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0.0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id or

## Análise de Valores Nulos

14. Qual a quantidade de valores nulos em cada feature?(0,66)
15. Qual a proporção de valores nulos por valores não nulos em cada feature?(0,66)
16. Faça o tratamento desses nulos e explique por que escolheu cada método.(0,68)

In [7]:
# Verificação de valores nulos no DataFrame
contagem_nulos = df_tratado.isnull().sum()

# Ordena a contagem de nulos em ordem decrescente para melhor visualização
print("\nContagem de valores nulos por feature:")
print(contagem_nulos.sort_values(ascending=False))

contagem_nao_nulos = df_tratado.notnull().sum()
proporcao = contagem_nulos / contagem_nao_nulos
proporcao_ordenada = proporcao.sort_values(ascending=False)
print("\nProporção de valores nulo por valores não nulos (Ordenado por Maiores):")
print(proporcao_ordenada)    



Contagem de valores nulos por feature:
belongs_to_collection    40970
homepage                 37682
tagline                  25051
genero_principal          2442
overview                   954
                         ...  
Uzbekistan                   0
Venezuela                    0
Vietnam                      0
Yugoslavia                   0
Zimbabwe                     0
Length: 186, dtype: int64

Proporção de valores nulo por valores não nulos (Ordenado por Maiores):
belongs_to_collection    9.118629
homepage                 4.842822
tagline                  1.227268
genero_principal         0.056763
overview                 0.021434
                           ...   
Uzbekistan               0.000000
Venezuela                0.000000
Vietnam                  0.000000
Yugoslavia               0.000000
Zimbabwe                 0.000000
Length: 186, dtype: float64


In [ ]:
# --- 1. Tratamento das colunas com muitos nulos ---

# Lista de colunas a serem removidas,m pois as que possuem a maior proporção de nulos
colunas_para_remover = ['belongs_to_collection', 'homepage']

# Verificando se as colunas existem antes de tentar removê-las
colunas_existentes = [col for col in colunas_para_remover if col in df_tratado.columns]

if colunas_existentes:
    df_limpeza = df_tratado.drop(columns=colunas_existentes)
    print(f"Colunas removidas com sucesso: {colunas_existentes}")
else:
    df_limpeza = df_tratado.copy() # Cria uma cópia para manter o padrão
    print("Nenhuma das colunas especificadas para remoção foi encontrada.")

# --- 2. Tratamento da coluna 'budget' com poucos nulos ---

# Verificando se a coluna 'budget' existe
if 'budget' in df_limpeza.columns:
    # Calcula a mediana da coluna 'budget'
    # O .dropna() garante que os nulos não interfiram no cálculo
    mediana_budget = df_limpeza['budget'].median()

    # Preenche os valores nulos (NaN) com a mediana calculada
    df_limpeza['budget'] = df_limpeza['budget'].fillna(mediana_budget)
    
    print(f"\nValores nulos da coluna 'budget' preenchidos com a mediana: {mediana_budget:.2f}")
else:
    print("\nA coluna 'budget' não foi encontrada para tratamento.")


# --- 3. Verificação Final ---
print("\nVerificação da quantidade de nulos após o tratamento:")
print(df_limpeza['budget'].isnull().sum())

# Agora o DataFrame 'df_tratado' está pronto para as próximas etapas!


Colunas removidas com sucesso: ['belongs_to_collection', 'homepage']

Valores nulos da coluna 'budget' preenchidos com a mediana: 0.00

Verificação da quantidade de nulos após o tratamento:
0


## Visualização Gráfica

17. Visualização Gráfica da feature `state`.(2)

In [57]:
#df.head(100)
print(df.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'generos_nomes', 'genero_principal'],
      dtype='object')
